In [59]:

from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
data = pd.read_csv("../data.csv")
print(data.shape, len(data))

label = 'is_norm'
presets = 'best_quality'
time_limit = 60*30
auto_stack = True
dynamic_stacking = False
num_stack_levels = None
num_bag_folds = None
num_bag_sets = None
excluded_model_types = ["KNN"]

(21826, 33) 21826


In [68]:
data[data["age"]>=96]

,is_norm,age,sex,height,weight,nurse,site,device,heart_axis,average_rr_interval_lead_0,...,average_rr_interval_lead_7,variance_rr_interval_lead_7,average_rr_interval_lead_8,variance_rr_interval_lead_8,average_rr_interval_lead_9,variance_rr_interval_lead_9,average_rr_interval_lead_10,variance_rr_interval_lead_10,average_rr_interval_lead_11,variance_rr_interval_lead_11


In [52]:
data.columns

Index(['is_norm', 'age', 'sex', 'height', 'weight', 'nurse', 'site', 'device',
       'heart_axis', 'average_rr_interval_lead_0',
       'variance_rr_interval_lead_0', 'average_rr_interval_lead_1',
       'variance_rr_interval_lead_1', 'average_rr_interval_lead_2',
       'variance_rr_interval_lead_2', 'average_rr_interval_lead_3',
       'variance_rr_interval_lead_3', 'average_rr_interval_lead_4',
       'variance_rr_interval_lead_4', 'average_rr_interval_lead_5',
       'variance_rr_interval_lead_5', 'average_rr_interval_lead_6',
       'variance_rr_interval_lead_6', 'average_rr_interval_lead_7',
       'variance_rr_interval_lead_7', 'average_rr_interval_lead_8',
       'variance_rr_interval_lead_8', 'average_rr_interval_lead_9',
       'variance_rr_interval_lead_9', 'average_rr_interval_lead_10',
       'variance_rr_interval_lead_10', 'average_rr_interval_lead_11',
       'variance_rr_interval_lead_11'],
      dtype='object')

In [53]:
data

,is_norm,age,sex,height,weight,nurse,site,device,heart_axis,average_rr_interval_lead_0,...,average_rr_interval_lead_7,variance_rr_interval_lead_7,average_rr_interval_lead_8,variance_rr_interval_lead_8,average_rr_interval_lead_9,variance_rr_interval_lead_9,average_rr_interval_lead_10,variance_rr_interval_lead_10,average_rr_interval_lead_11,variance_rr_interval_lead_11
0,1,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,NaN,93.666667,...,93.900000,3.290000,93.800000,2.960000,93.555556,2.691358,93.666667,2.000000,93.555556,2.691358
1,1,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,NaN,127.166667,...,127.166667,57.805556,127.166667,57.805556,127.000000,57.333333,127.000000,55.333333,127.000000,57.333333
2,1,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,NaN,94.000000,...,94.111111,4.320988,94.111111,3.432099,94.111111,4.320988,94.111111,4.320988,94.111111,4.320988
3,1,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,NaN,80.636364,...,80.636364,13.867769,80.636364,12.413223,80.636364,13.867769,80.636364,13.504132,80.636364,13.504132
4,1,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,NaN,90.500000,...,90.500000,24.050000,90.500000,24.250000,90.500000,22.650000,90.500000,24.050000,90.500000,24.050000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21821,0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,LAD,59.466667,...,55.687500,24.964844,55.750000,21.312500,55.687500,24.964844,55.687500,24.839844,55.687500,24.714844
21822,1,93.0,0,NaN,NaN,1.0,2.0,AT-60 3,MID,89.555556,...,89.444444,1.135802,89.444444,1.135802,89.444444,1.135802,89.444444,1.135802,89.444444,1.135802
21823,0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,MID,102.250000,...,102.250000,15.687500,102.250000,15.687500,102.250000,16.937500,102.250000,16.687500,102.250000,17.687500
21824,1,64.0,1,NaN,NaN,1.0,2.0,AT-60 3,LAD,85.900000,...,85.800000,101.760000,85.800000,106.560000,85.800000,106.960000,85.800000,106.960000,85.900000,107.490000


In [54]:

cols_to_keep = data.filter(like='lead', axis=1).columns

# add label to the list of columns to keep
cols_to_keep = list(cols_to_keep) + [label] + ["device", "sex", "age"]


# # Get a list of all columns that contain the word 'lead'
# cols_to_drop = data.filter(like='lead', axis=1).columns
# # Drop these columns from the dataframe
# data = data.drop(columns=cols_to_drop)


# keep only the columns the cols_to_keep
data = data[cols_to_keep]
data


,average_rr_interval_lead_0,variance_rr_interval_lead_0,average_rr_interval_lead_1,variance_rr_interval_lead_1,average_rr_interval_lead_2,variance_rr_interval_lead_2,average_rr_interval_lead_3,variance_rr_interval_lead_3,average_rr_interval_lead_4,variance_rr_interval_lead_4,...,average_rr_interval_lead_9,variance_rr_interval_lead_9,average_rr_interval_lead_10,variance_rr_interval_lead_10,average_rr_interval_lead_11,variance_rr_interval_lead_11,is_norm,device,sex,age
0,93.666667,1.777778,93.555556,2.469136,104.333333,796.444444,93.888889,22.987654,93.666667,1.777778,...,93.555556,2.691358,93.666667,2.000000,93.555556,2.691358,1,CS-12 E,1,56.0
1,127.166667,56.472222,127.000000,55.333333,127.166667,56.805556,127.000000,57.333333,127.166667,57.805556,...,127.000000,57.333333,127.000000,55.333333,127.000000,57.333333,1,CS-12 E,0,19.0
2,94.000000,3.555556,94.111111,4.320988,93.444444,14.024691,94.000000,20.000000,94.000000,3.333333,...,94.111111,4.320988,94.111111,4.320988,94.111111,4.320988,1,CS-12 E,1,37.0
3,80.636364,13.867769,80.636364,12.776860,80.636364,13.504132,80.545455,13.338843,80.545455,12.975207,...,80.636364,13.867769,80.636364,13.504132,80.636364,13.504132,1,CS-12 E,0,24.0
4,90.500000,25.050000,90.500000,24.050000,90.600000,24.240000,91.300000,66.610000,90.600000,25.440000,...,90.500000,22.650000,90.500000,24.050000,90.500000,24.050000,1,CS-12 E,1,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21821,59.466667,197.315556,55.687500,24.839844,55.312500,36.464844,55.687500,26.339844,55.750000,11.812500,...,55.687500,24.964844,55.687500,24.839844,55.687500,24.714844,0,AT-60 3,1,67.0
21822,89.555556,1.135802,89.444444,1.135802,89.555556,11.580247,89.444444,1.135802,89.555556,0.913580,...,89.444444,1.135802,89.444444,1.135802,89.444444,1.135802,1,AT-60 3,0,93.0
21823,102.250000,16.687500,102.250000,17.687500,47.944444,849.496914,102.250000,14.687500,102.250000,16.937500,...,102.250000,16.937500,102.250000,16.687500,102.250000,17.687500,0,AT-60 3,1,59.0
21824,85.900000,107.490000,85.900000,107.490000,85.800000,196.160000,85.800000,108.560000,85.900000,109.890000,...,85.800000,106.960000,85.800000,106.960000,85.900000,107.490000,1,AT-60 3,1,64.0


In [55]:
train_test_split = 0.8

train_data, test_data = data[0:int(len(data)*train_test_split)], data[int(len(data)*train_test_split):]

print(f"Unique classes: {list(train_data[label].unique())}")

# print number of samples in each class
print(train_data[label].value_counts())

Unique classes: [1, 0]
is_norm
0    9559
1    7901
Name: count, dtype: int64


In [56]:
predictor = TabularPredictor(label=label).fit(train_data, time_limit=time_limit, presets=presets, auto_stack=auto_stack, num_stack_levels=num_stack_levels, num_bag_folds=num_bag_folds, num_bag_sets=num_bag_sets, excluded_model_types=excluded_model_types, dynamic_stacking=dynamic_stacking)

No path specified. Models will be saved in: "AutogluonModels/ag-20240221_080013"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=5
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "AutogluonModels/ag-20240221_080013"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.3.0: Wed Dec 20 21:30:44 PST 2023; root:xnu-10002.81.5~7/RELEASE_ARM64_T6000
CPU Count:          8
Memory Avail:       4.80 GB / 16.00 GB (30.0%)
Disk Space Avail:   14.05 GB / 460.43 GB (3.1%)
Train Data Rows:    17460
Train Data Columns: 27
Label Column:       is_norm
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually s

In [57]:
predictor.evaluate(test_data)

{'accuracy': 0.7437013284470911,
 'balanced_accuracy': 0.7044123324444923,
 'mcc': 0.433882916658368,
 'roc_auc': 0.8053557159659188,
 'f1': 0.6153317291165348,
 'precision': 0.6975837879968823,
 'recall': 0.550430504305043}